In [ ]:
import shutil, subprocess, tempfile
from pathlib import Path

# --- Locate src/data from a notebook in notebooks/ (siblings) ---
NB_DIR   = Path.cwd().resolve()
SRC_DIR  = NB_DIR.parent / "src"
DATA_DIR = SRC_DIR / "data"

print("Notebook CWD:", NB_DIR)
print("SRC_DIR     :", SRC_DIR)
print("DATA_DIR    :", DATA_DIR)

if not SRC_DIR.exists() or not DATA_DIR.exists():
    raise FileNotFoundError("Could not find sibling src/ and src/data/ next to notebooks/.")

# --- Files to test (must live in src/data) ---
mapping_files = [
    "bfo-mapping.ttl",
    "ies-mapping.ttl",
    "ccom-mapping.ttl",
    "qudt-mapping.ttl",
    "ccot-mapping.ttl",
    "time-mapping.ttl",  # keep this name if that's what your course uses
]

# --- Sanity: is ROBOT available? ---
if shutil.which("robot") is None:
    raise RuntimeError("ROBOT not found on PATH. Install ROBOT and make sure 'robot' is runnable.")

# Print robot version (optional; comment out if you prefer silence)
subprocess.run(["robot", "--version"], check=False)

reasoners = ["HermiT", "ELK"]

def run_robot_reason(input_file: Path, reasoner: str) -> bool:
    """Run robot reason with a temporary output; return True on success."""
    # robot requires an --output, so use a temp file and let it vanish
    with tempfile.NamedTemporaryFile(suffix=".ttl", delete=True) as tmp:
        res = subprocess.run(
            [
                "robot", "reason",
                "--reasoner", reasoner,
                "--input", str(input_file),
                "--output", tmp.name,
            ],
            capture_output=True,
            text=True,
        )
    return res.returncode == 0

any_ran = False
for name in mapping_files:
    path = DATA_DIR / name
    if not path.exists():
        print(f"SKIP  (missing): {name}")
        continue
    any_ran = True
    for r in reasoners:
        ok = run_robot_reason(path, r)
        status = "SUCCESS" if ok else "FAILURE"
        print(f"{status}: {name} [{r}]")

if not any_ran:
    print("No mapping files found in src/data to test.") 
